In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownTextSplitter

In [13]:

from langchain_community.embeddings import HuggingFaceEmbeddings

In [15]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep
import time
from bs4 import BeautifulSoup
import re

In [16]:
URL = "https://api.hh.ru/vacancies"

In [17]:
job_titles_list = ["data scientist", "data analyst"]

In [18]:
base_params_template = {
    'area': 113,           # Регион: Россия
    'per_page': 100,       
    'order_by': 'publication_time' 
}

In [19]:
all_vacancies_data = []

In [25]:
MAX_PAGES = 4

In [27]:
def download_vacancy(base_params, page_num):
    """
    Выполняет HTTP-запрос к API HH.ru с обновлением номера страницы.
    """
    # 1. КОПИРУЕМ базовый шаблон и обновляем номер страницы (КЛЮЧЕВОЙ ШАГ!)
    current_params = base_params.copy() 
    current_params["page"] = page_num 
    
    try:
        response = requests.get(URL, params=current_params)
        response.raise_for_status() # Вызывает исключение для 4xx/5xx ошибок
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка HTTP на странице {page_num}: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Ошибка декодирования JSON на странице {page_num}")
        return None

In [29]:
def get_full_details(vacancy_id):
    """
    Получает полное описание вакансии по ее ID.
    """
    detail_url = f"https://api.hh.ru/vacancies/{vacancy_id}"
    try:
        response = requests.get(detail_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка получения деталей для ID {vacancy_id}: {e}")
        return None

In [31]:
# 3. ФУНКЦИЯ ОБРАБОТКИ ДАННЫХ (Сплющивание)
def process_vacancy(vacancy, search_term):
    salary_data = vacancy.get('salary')
    min_salary = None
    if salary_data and isinstance(salary_data, dict) and salary_data.get('from'):
        min_salary = salary_data['from']
        
    # 2. Работодатель: вложенное поле
    employer_info = vacancy.get('employer', {})
    employer_name = employer_info.get('name', 'N/A')
    
    # 3. Город: вложенное поле
    area_info = vacancy.get('area', {})
    city_name = area_info.get('name', 'N/A')
    
    # 4. Требования и обязанности (извлекаем чистый текст)
    snippet = vacancy.get('snippet', {})
    requirement = snippet.get('requirement', '')
    responsibility = snippet.get('responsibility', '')
    
    # 5. Опыт: вложенное поле
    experience_info = vacancy.get('experience', {})
    experience_name = experience_info.get('name', 'N/A')

    # 1. Получаем ID вакансии
    vacancy_id = vacancy.get('id')
    full_description = "N/A"
    
    if vacancy_id:
        # 2. Делаем новый запрос за деталями
        full_data = get_full_details(vacancy_id)
        
        # 3. Извлекаем поле 'description' (там HTML-код)
        if full_data and full_data.get('description'):
            full_description = full_data['description']
        
        # ОЧЕНЬ ВАЖНО: Добавляем небольшую паузу после каждого запроса деталей
        time.sleep(0.1)

    # Создание плоского словаря
    flat_row = {
        'full_description': full_description,
        'search_term': search_term, # Какую должность мы искали
        'vacancy_id': vacancy.get('id'),
        'vacancy_name': vacancy.get('name'),
        'city_name': city_name,
        'min_salary': min_salary,
        'employer_name': employer_name,
        'published_at': vacancy.get('published_at'),
        'experience': experience_name,
        'schedule': vacancy.get('schedule', {}).get('name'),
        'employment': vacancy.get('employment', {}).get('name'),
        'requirement': requirement,
        'responsibility': responsibility,
        # Добавьте другие поля по необходимости
    }
    return flat_row

In [33]:
# 4. ОСНОВНОЙ ЦИКЛ ПАРСИНГА
for job in job_titles_list:
    print(f"--- Начинаем парсинг для: '{job}' ---")
    
    # 1. Определяем параметры для текущей профессии (добавляем 'text')
    current_search_params = base_params_template.copy()
    current_search_params['text'] = job
    
    for page in range(MAX_PAGES):
        vac_data = download_vacancy(current_search_params, page)
        
        # Если загрузка не удалась (вернула None или пустой словарь)
        if not vac_data:
            print(f"Пропуск страницы {page} из-за ошибки запроса.")
            continue # Переходим к следующей итерации цикла
        
        # БЕЗОПАСНАЯ ПРОВЕРКА (устраняет KeyError, как мы обсуждали)
        if 'items' in vac_data:
            vacancies_on_page = vac_data['items']
            
            if not vacancies_on_page:
                print(f"Страница {page} пуста. Завершаем сбор для '{job}'.")
                break # Выходим из цикла по страницам, так как вакансии закончились
            
            for vacancy in vacancies_on_page:
                # 2. Сплющиваем и добавляем плоский словарь в финальный список
                flat_row = process_vacancy(vacancy, job)
                all_vacancies_data.append(flat_row)
                
            print(f"Обработано {len(vacancies_on_page)} вакансий на странице {page}. Всего: {len(all_vacancies_data)}")
            
        else:
            print(f"Ошибка структуры данных на странице {page}: Нет ключа 'items'.")
            
        time.sleep(0.2) # Пауза между запросами

--- Начинаем парсинг для: 'data scientist' ---
Обработано 100 вакансий на странице 0. Всего: 100
Обработано 100 вакансий на странице 1. Всего: 200
Обработано 100 вакансий на странице 2. Всего: 300
Обработано 20 вакансий на странице 3. Всего: 320
--- Начинаем парсинг для: 'data analyst' ---
Обработано 100 вакансий на странице 0. Всего: 420
Обработано 100 вакансий на странице 1. Всего: 520
Обработано 100 вакансий на странице 2. Всего: 620
Обработано 100 вакансий на странице 3. Всего: 720


In [34]:
# 5. СОЗДАНИЕ DATAFRAME И CSV
print("\n--- Завершение сбора данных ---")
print(f"Общее количество собранных записей: {len(all_vacancies_data)}")

if all_vacancies_data:
    df = pd.DataFrame(all_vacancies_data)
    
    # Заполняем пустые значения (None) в зарплате нулем для удобства анализа
    df['min_salary'] = df['min_salary'].fillna(0) 
    
    filename = 'hh_vacancies_data.csv'
    # index=False исключает столбец с индексами из CSV
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"\n✅ Датасет успешно создан и сохранен в файл: {filename}")
else:
    print("❌ Список вакансий пуст. Файл CSV не создан.")


--- Завершение сбора данных ---
Общее количество собранных записей: 720

✅ Датасет успешно создан и сохранен в файл: hh_vacancies_data.csv


In [35]:
df.head()

,full_description,search_term,vacancy_id,vacancy_name,city_name,min_salary,employer_name,published_at,experience,schedule,employment,requirement,responsibility
0,"<p>Требуется разработать агента, который будет...",data scientist,127489893,Data Scientist (Управление моделирования и исс...,Москва,0.0,СБЕР,2025-11-10T17:08:11+0300,От 3 до 6 лет,Полный день,Полная занятость,Опыт работы <highlighttext>Data</highlighttext...,Сбор требований по моделям и коммуникация с за...
1,"<b>Обязанности:</b><br />-Разрабатывать, внедр...",data scientist,126119482,Data Scientist Middle/Middle+,Москва,0.0,Альфа-Банк,2025-11-10T17:07:29+0300,От 1 года до 3 лет,Полный день,Полная занятость,Уверенное владение Python для промышленной раз...,"Разрабатывать, внедрять и оптимизировать класс..."
2,<p>Если ТЕБЕ нравятся крутые и амбициозные зад...,data scientist,127117611,Аналитик данных,Нижний Новгород,40000.0,КСК-Эйч Ар,2025-11-10T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Расширение технических знаний и развитие из ан...,Автоматизация выгрузки и сбора данных. - Ad/ho...
3,<p>Если ТЕБЕ нравятся крутые и амбициозные зад...,data scientist,127117612,Аналитик данных,Самара,40000.0,КСК-Эйч Ар,2025-11-10T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Расширение технических знаний и развитие из ан...,Автоматизация выгрузки и сбора данных. - Ad/ho...
4,<p>​​​Наша команда каждый день работает над по...,data scientist,126737674,Старший бизнес-аналитик CLTV,Москва,0.0,билайн,2025-11-10T15:20:32+0300,От 3 до 6 лет,Удаленная работа,Полная занятость,"Высшее образование в сфере экономики, математи...",...пилоты и А/В тесты. Оформлять инсайты в пре...


In [36]:
df['employer_name'].value_counts()

employer_name
СБЕР                50
Ozon                22
билайн              16
Альфа-Банк          13
X5 Tech             11
                    ..
Evercode Lab         1
Cloud.ru             1
АльфаСтрахование     1
Мэлон Фэшн Груп      1
Инженеры продаж      1
Name: count, Length: 416, dtype: int64

In [37]:
df['full_description'][0]

'<p>Требуется разработать агента, который будет оркестрировать несколько ML для анализа биржевых стаканов облигаций и на основании этого предлагает оптимальные варианты для совершения сделок покупки/продажи бумаг трейдерам.</p> <h3><strong>Обязанности</strong></h3> <ul> <li>сбор требований по моделям и коммуникация с заказчиком и DE</li> <li>определение целевых метрик решения и согласование их с заказчиком</li> <li>сбор и подготовка данных (Hadoop, pyspark, написание парсеров для сборки внешних данных)</li> <li>управление ресурсом Middle DS</li> <li>построение моделей по табличным данным</li> <li>поиск аномалий во временных рядах</li> <li>построение моделей предсказания временных рядов</li> <li>классификация новостей по степени влияния на рынок облигаций</li> <li>генерации торговых сигналов на основе неструктурированных данных</li> <li>создание RAG базы для генерации текстовых обоснований рекомендаций для трейдеров</li> <li>адаптация LLM для формулировки торговых идей на естественном я

In [38]:
#число уникальных описаний
unique_description = df['full_description'].nunique()
unique_description

670

In [39]:
unique_employers = df['employer_name'].nunique()
total_vacancies = len(df)
print(f"Общее количество вакансий: {total_vacancies}")
print(f"Количество уникальных работодателей: {unique_employers}")

Общее количество вакансий: 720
Количество уникальных работодателей: 416


In [40]:
top_employers = df['employer_name'].value_counts().head(10)
print("\nТоп-10 работодателей:")
print(top_employers)


Топ-10 работодателей:
employer_name
СБЕР           50
Ozon           22
билайн         16
Альфа-Банк     13
X5 Tech        11
Т-Банк         11
МТС            11
WILDBERRIES     9
Aston           7
LIAN            6
Name: count, dtype: int64


In [41]:
df_alt = df.copy()

создам порог и почищу датасет от нулевых значений

In [43]:
df_alt.notnull().sum()

full_description    720
search_term         720
vacancy_id          720
vacancy_name        720
city_name           720
min_salary          720
employer_name       720
published_at        720
experience          720
schedule            720
employment          720
requirement         715
responsibility      718
dtype: int64

In [44]:
#df_alt = df_alt.loc[:, df_alt.notnull().sum() >= treshhold]
#df_alt.info()

In [45]:
df_alt.head(2)

,full_description,search_term,vacancy_id,vacancy_name,city_name,min_salary,employer_name,published_at,experience,schedule,employment,requirement,responsibility
0,"<p>Требуется разработать агента, который будет...",data scientist,127489893,Data Scientist (Управление моделирования и исс...,Москва,0.0,СБЕР,2025-11-10T17:08:11+0300,От 3 до 6 лет,Полный день,Полная занятость,Опыт работы <highlighttext>Data</highlighttext...,Сбор требований по моделям и коммуникация с за...
1,"<b>Обязанности:</b><br />-Разрабатывать, внедр...",data scientist,126119482,Data Scientist Middle/Middle+,Москва,0.0,Альфа-Банк,2025-11-10T17:07:29+0300,От 1 года до 3 лет,Полный день,Полная занятость,Уверенное владение Python для промышленной раз...,"Разрабатывать, внедрять и оптимизировать класс..."


Очищу датасет до минимума необходимой информации

In [47]:
#features_to_save = ['id', 'name', 'full_description', 'snippet.responsibility', 'snippet.requirement',
                    #'schedule.name', 'experience.name', 'employment.name', 'address.city']
#df_final = df_alt[features_to_save]

In [48]:
#df_final.fillna('', inplace=True)

In [49]:
df_alt.fillna('', inplace=True)

очищу данные от html символов, пунктуации и тд

In [51]:
import nltk
import unicodedata

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import stanza

In [52]:
#nltk.download('stopwords')
#nltk.download('punkt_tab')

In [53]:
nlp_ru = stanza.Pipeline(lang='ru', processors='tokenize,lemma')
russian_stopwords = set(stopwords.words('russian'));

2025-11-11 12:08:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-11-11 12:08:13 INFO: Downloaded file to C:\Users\User\stanza_resources\resources.json
2025-11-11 12:08:13 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| lemma     | syntagrus_nocharlm |

2025-11-11 12:08:13 INFO: Using device: cpu
2025-11-11 12:08:13 INFO: Loading: tokenize
2025-11-11 12:08:13 INFO: Loading: lemma
2025-11-11 12:08:16 INFO: Done loading processors!


In [54]:
def clean_text_simple(input_text, stop_words_set=None):
    """
    Упрощенная улучшенная версия без лемматизации
    """
    if pd.isna(input_text) or input_text is None:
        return ""

    text = str(input_text)
    
    # 1. Удаляем HTML
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # 2. Удаляем URL
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    
    # 3. Сохраняем базовую структуру текста
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9\s\-_\.]', ' ', text)
    
    # 4. Нижний регистр
    text = text.lower()
    
    # 5. Обрабатываем составные слова
    text = re.sub(r'(\b\w+)-(\w+\b)', r'\1_\2', text)
    
    # 6. Токенизация
    words = text.split()
    
    # 7. Фильтрация
    if stop_words_set:
        filtered_words = [
            word for word in words 
            if (word not in stop_words_set and 
                len(word) > 1 and
                not word.isdigit())  # убираем чистые числа
        ]
    else:
        filtered_words = [word for word in words if len(word) > 1 and not word.isdigit()]
    
    return ' '.join(filtered_words)

In [55]:
# Функция для очистки текста
'''def clean_and_lemmatize_russian(input_text, nlp_pipeline, stop_words_set):    
    if pd.isna(input_text) or input_text is None:
        return ""

    clean_text = str(input_text)
    # HTML-теги
    clean_text = re.sub('<[^<]+?>', '', clean_text)
    
    # URL и ссылки
    clean_text = re.sub(r'http\S+', '', clean_text)

    # Email
    clean_text = re.sub(r'\S+@\S+', '', clean_text)

    clean_text = clean_text.lower()

    # Заменяем нежелательные символы на пробелы, разрешаем буквы, цифры и символы: -_+=/&
    clean_text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9\-_+=/&]', ' ', clean_text)

    # 5. Разделение на слова и удаление стоп-слов
    words = clean_text.split(' ')

    # Убираем все пробелы
    clean_text = re.sub('\s+', ' ', clean_text)
    
    # Фильтруем слова, убирая стоп-слова и пустые строки (возникшие при очистке)
    filtered_words: List[str] = [
        word for word in words 
        if word and word not in stop_words_set
    ]
    
    # Возвращаем очищенный текст, состоящий из оригинальных слов (без лемматизации)
    return ' '.join(filtered_words)'''

'def clean_and_lemmatize_russian(input_text, nlp_pipeline, stop_words_set):    \n    if pd.isna(input_text) or input_text is None:\n        return ""\n\n    clean_text = str(input_text)\n    # HTML-теги\n    clean_text = re.sub(\'<[^<]+?>\', \'\', clean_text)\n\n    # URL и ссылки\n    clean_text = re.sub(r\'http\\S+\', \'\', clean_text)\n\n    # Email\n    clean_text = re.sub(r\'\\S+@\\S+\', \'\', clean_text)\n\n    clean_text = clean_text.lower()\n\n    # Заменяем нежелательные символы на пробелы, разрешаем буквы, цифры и символы: -_+=/&\n    clean_text = re.sub(r\'[^a-zA-Zа-яА-ЯёЁ0-9\\-_+=/&]\', \' \', clean_text)\n\n    # 5. Разделение на слова и удаление стоп-слов\n    words = clean_text.split(\' \')\n\n    # Убираем все пробелы\n    clean_text = re.sub(\'\\s+\', \' \', clean_text)\n\n    # Фильтруем слова, убирая стоп-слова и пустые строки (возникшие при очистке)\n    filtered_words: List[str] = [\n        word for word in words \n        if word and word not in stop_words_set\n 

In [60]:
text = 'О компании Мы — успешный fashion-бренд с 15-летней историей и собственным производством женской одежды. Полный цикл: от закупки ткани до продажи на маркетплейсах. Наш основной оборот идёт через Wildberries и Ozon (входим в топ-1% продавцов одежды на Ozon ). Сейчас мы запускаем проект по построению аналитической/рекомендательной платформы с нуля . Цель — сделать управление ассортиментом, ценами, закупками, производством и рекламой максимально прозрачным и основанным на данных . Если внутренняя система покажет результат, мы планируем вывести её на рынок как отдельный SaaS-продукт для других продавцов маркетплейсов. Это возможность присоединиться на старте и влиять на архитектуру продукта.(уже есть заинтересованные селлеры) Задачи 1. Сбор данных и интеграции Подключение API Wildberries, Ozon и других площадок. Получение данных: Продажи (выручка, заказы, выкупы, возвраты, статусы). Остатки на складах, логистика. Рекламные метрики (показы, клики, CTR, CPC, CPA, ROI). Цены, динамика скидок, участие в акциях. Позиции в поиске, отзывы, рейтинг. Интеграция данных из 1С (по ODATA) . Загрузка данных из Excel/CSV. Парсинг маркетплейсов для конкурентного анализа. 2. Построение хранилища данных Проектирование и реализация Data LakeHouse . Хранение данных в S3 или аналогах , первичная обработка. Создание структуры данных (сырые → очищенные → витрины). Использование ClickHouse или других колоночных БД. Настройка базовых ETL/ELT-процессов . 3. Аналитика и прогнозирование Прогноз спроса и продаж (по товарам и категориям). Оптимизация остатков и распределение поставок по складам WB/Ozon . Автоматизация рекламных кампаний: динамические ставки, удаление неэффективных ключей и кластеров, анализ CTR/конверсий. Расчёт маржинальности и прибыли. Оценка эффективности SKU, поставок и логистики. 4. Визуализация и отчётность Построение дашбордов в Yandex DataLens . Создание таблиц и аналитических отчётов. Возможна разработка части аналитики в самописном веб-приложении . Требования Опыт работы с данными: ETL, SQL, Python . Знание API: работа с REST API маркетплейсов, интеграция с 1С ODATA. Опыт с БД: PostgreSQL, ClickHouse (или аналогами). Инструменты: Docker, Git . Библиотеки: pandas, requests, airflow (или опыт других пайплайн-менеджеров). BI: DataLens (обязательно), понимание метрик аналитики. Умение строить прогнозы (time series, ML — плюс). Умение самостоятельно доводить задачи до результата. Плюсом будет Опыт проектирования Data LakeHouse. Опыт работы с маркетплейсами (WB/Ozon). Знание MLOps. Опыт работы в e-commerce или производстве. Навыки FastAPI/Flask для интеграций и сервисов. Мы предлагаем Участие в проекте с нуля — ключевая роль. Возможность влиять на архитектуру и решения. Рост вместе с продуктом. Гибкий график, гибридный формат (после ИС — частично удалёнка). Реальная свобода выбора технологий и решений. Долгосрочная работа над продуктом, а не «таск-менеджмент». Важно — перед откликом Мы ищем самостоятельного специалиста , а не стажёра или начинающего уровня. На старте у нас нет ресурсов учить с нуля , поэтому: ❗ Если вы не умеете или не готовы быстро научиться: работать с API WB/Ozon, ETL, Python, работать с ClickHouse/PostgreSQL, строить дашборды в DataLens(или других системах) — — не откликайтесь на вакансию. В сопроводительном письме укажите: Ваш уровень по навыкам: Python / SQL / API / ClickHouse / DataLens (по 10-балльной шкале). Реальный опыт: 2–3 проекта или задачи, которыми вы гордитесь. Желательный уровень дохода и формат работы (офис/гибрид). — на испытательный срок мы ищем сотрудников исключительно в офис!!! , не откликайтесь на вакансию если это вас не устраивает! p.s.: Мы не хотим делать копию существующих на рынке продуктов, мы тестировали многие, но они не отвечают нашим задачам и задачам селлеров с кем мы знакомы, у нас уже есть некоторые наработки, поэтому это скорее гибрид нескольких продуктов, чтобы максимально автоматизировать рутинные задачи и быстро принимать управленческие решения.'

In [78]:
cleaned_text = clean_text_simple(text,  russian_stopwords)

In [82]:
print(cleaned_text)

компании успешный fashion_бренд 15_летней историей собственным производством женской одежды. полный цикл закупки ткани продажи маркетплейсах. наш основной оборот идёт wildberries ozon входим топ_1 продавцов одежды ozon запускаем проект построению аналитической рекомендательной платформы нуля цель сделать управление ассортиментом ценами закупками производством рекламой максимально прозрачным основанным данных внутренняя система покажет результат планируем вывести её рынок отдельный saas_продукт других продавцов маркетплейсов. это возможность присоединиться старте влиять архитектуру продукта. заинтересованные селлеры задачи 1. сбор данных интеграции подключение api wildberries ozon других площадок. получение данных продажи выручка заказы выкупы возвраты статусы остатки складах логистика. рекламные метрики показы клики ctr cpc cpa roi цены динамика скидок участие акциях. позиции поиске отзывы рейтинг. интеграция данных 1с odata загрузка данных excel csv. парсинг маркетплейсов конкурентног

Обработаю текстовые фичи функцией для удаления html, пунктуации, приведения кстандартному виду

In [93]:
df_alt['description_cleaned'] = df_alt['full_description'].apply(
    lambda text: clean_text_simple(text, russian_stopwords)
)

In [95]:
df_alt['requirement_cleaned'] = df_alt['requirement'].apply(
    lambda text: clean_text_simple(text, russian_stopwords))

In [97]:
df_alt['requirement_cleaned'][1]

'уверенное владение python промышленной разработки. опыт работы pyspark sql обработки больших данных. глубокие знания классических методов машинного...'

In [99]:
df_final = df_alt.copy()

In [101]:
df_final = df_final.drop(['full_description', 'requirement'], axis=1)

In [103]:
df_final.head()

,search_term,vacancy_id,vacancy_name,city_name,min_salary,employer_name,published_at,experience,schedule,employment,responsibility,description_lemmatized,requirement_lemmatized,description_cleaned,requirement_cleaned
0,data scientist,127489893,Data Scientist (Управление моделирования и исс...,Москва,0.0,СБЕР,2025-11-10T17:08:11+0300,От 3 до 6 лет,Полный день,Полная занятость,Сбор требований по моделям и коммуникация с за...,требуется разработать агента который оркестрир...,опыт работы data scientist лет. образование на...,требуется разработать агента который оркестрир...,опыт работы data scientist лет. образование на...
1,data scientist,126119482,Data Scientist Middle/Middle+,Москва,0.0,Альфа-Банк,2025-11-10T17:07:29+0300,От 1 года до 3 лет,Полный день,Полная занятость,"Разрабатывать, внедрять и оптимизировать класс...",обязанности -разрабатывать внедрять оптимизиро...,уверенное владение python промышленной разрабо...,обязанности -разрабатывать внедрять оптимизиро...,уверенное владение python промышленной разрабо...
2,data scientist,127117611,Аналитик данных,Нижний Новгород,40000.0,КСК-Эйч Ар,2025-11-10T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Автоматизация выгрузки и сбора данных. - Ad/ho...,тебе нравятся крутые амбициозные задачи мечтае...,расширение технических знаний развитие аналити...,тебе нравятся крутые амбициозные задачи мечтае...,расширение технических знаний развитие аналити...
3,data scientist,127117612,Аналитик данных,Самара,40000.0,КСК-Эйч Ар,2025-11-10T16:55:15+0300,Нет опыта,Удаленная работа,Полная занятость,Автоматизация выгрузки и сбора данных. - Ad/ho...,тебе нравятся крутые амбициозные задачи мечтае...,расширение технических знаний развитие аналити...,тебе нравятся крутые амбициозные задачи мечтае...,расширение технических знаний развитие аналити...
4,data scientist,126737674,Старший бизнес-аналитик CLTV,Москва,0.0,билайн,2025-11-10T15:20:32+0300,От 3 до 6 лет,Удаленная работа,Полная занятость,...пилоты и А/В тесты. Оформлять инсайты в пре...,наша команда каждый день работает повышением к...,высшее образование сфере экономики математики ...,наша команда каждый день работает повышением к...,высшее образование сфере экономики математики ...


Чанкование

In [105]:
df_final['meta_header'] = (
    'Вакансия: ' + df_final['vacancy_name'].fillna('') + '.'
    ' Город:  ' + df_final['city_name'].fillna('') + '.' +
    " Опыт: " + df_final['experience'].fillna('') + ". " +
    "График: " + df_final['schedule'].fillna('') + ". " +
    "Занятость: " + df_final['employment'].fillna('') + ". "
    "Минимальная зарплата: " + df_final['min_salary'].fillna('').astype(str) + ".")
    

In [113]:
df_requirements = df_final.copy()
df_requirements['rag_chunk'] = (df_requirements['meta_header'] +
                                'Требования: ' + df_requirements['requirement_cleaned'].fillna('требования не указаны')+ '.'
                               " Обязанности: " + df_requirements['responsibility'].fillna('Обязанности не указаны.'))

In [115]:
rag_chunk_coloumn = df_requirements['rag_chunk']
df_final = pd.concat([df_final, rag_chunk_coloumn], axis=1)

In [117]:
df_requirements['rag_chunk'][0]

'Вакансия: Data Scientist (Управление моделирования и исследования данных). Город:  Москва. Опыт: От 3 до 6 лет. График: Полный день. Занятость: Полная занятость. Минимальная зарплата: 0.0.Требования: опыт работы data scientist лет. образование направлениям математика физика мат. методы экономике желательно выпускники мгу мфти.... Обязанности: Сбор требований по моделям и коммуникация с заказчиком и DE. Определение целевых метрик решения и согласование их с заказчиком. '

In [119]:
df_requirements = df_requirements[['vacancy_id', 'rag_chunk']]

In [121]:
df_requirements.head()

,vacancy_id,rag_chunk
0,127489893,Вакансия: Data Scientist (Управление моделиров...
1,126119482,Вакансия: Data Scientist Middle/Middle+. Город...
2,127117611,Вакансия: Аналитик данных. Город: Нижний Новг...
3,127117612,Вакансия: Аналитик данных. Город: Самара. Опы...
4,126737674,Вакансия: Старший бизнес-аналитик CLTV. Город:...


In [123]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

Применю различные методы чанкования и позже сравню качество по метрикам

In [126]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2");

In [127]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [133]:
#чанковние по приоритету разделителей
text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""] # Попытка разбить сначала по абзацам, потом по строкам, потом по пробелам
)

Далее использую продвинутые методы чанкования Fusion Retrieval и HyDE

In [136]:
fusion_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700, 
    chunk_overlap=70, 
    separators=["\n\n", "\n", ". ", " ", ""], 
    length_function=len
)

In [166]:
'''hyde_splitter = SemanticChunker(
    embeddings=embeddings,
    # Создавать разрывы в точках, где семантическое сходство ниже 75%
    breakpoint_threshold_type="percentile", 
    # threshold=75 (значение по умолчанию)
)'''

NameError: name 'SemanticChunker' is not defined

In [173]:
from typing import List, Dict, Any

In [179]:
def create_general_chunks(row: pd.Series, splitter: Any) -> List[Dict[str, Any]]:
   
    # 1. Извлечение необходимых данных
    meta_header = row['meta_header']
    description_text = row['description_cleaned']
    # Предполагается, что 'rag_chunk' уже создан и готов к использованию
    rag_chunk_text = row['rag_chunk'] 
    vacancy_id = row['vacancy_id']

    general_chunks_list: List[Dict[str, Any]] = []

    # --- A. Добавляем короткий, сводный чанк (целиком) ---
    # Этот чанк важен для точного поиска по ключевым атрибутам
    if pd.notna(rag_chunk_text) and rag_chunk_text.strip():
        # Добавляем метку, чтобы LLM понимала, что это "СВОДКА"
        final_rag_chunk_summary = "СВОДКА ВАКАНСИИ: " + rag_chunk_text.strip()
        
        general_chunks_list.append({
            'id': vacancy_id,
            'rag_chunk': final_rag_chunk_summary,
            'chunk_type': 'summary' 
        })
    
    if pd.notna(description_text) and description_text.strip():
        
        # 2. Разбиение: splitter (будь то Semantic или Recursive) создает объекты Document
        chunks_description: List[Document] = splitter.create_documents([description_text])
        
        # 3. Формирование финальных чанков из описания
        for chunk in chunks_description:
            chunk_text = chunk.page_content.strip()

            # Префикс (метаданные) + метка о типе чанка
            final_rag_chunk_detail = (
                meta_header + 
                " [ДЕТАЛЬНОЕ ОПИСАНИЕ]: " + chunk_text
            )
            
            general_chunks_list.append({
                'id': vacancy_id,
                'rag_chunk': final_rag_chunk_detail,
                'chunk_type': 'detail'
            })
            
    return general_chunks_list

In [181]:
all_general_chunks_recursive = []
for index, row in df_final.iterrows():
    #  Вызываем нашу функцию для получения списка чанков из одной строки
    chunks_for_row = create_general_chunks(row, splitter=text_splitter_recursive)
    #  Добавляем все чанки из текущей строки в общий список
    all_general_chunks_recursive.extend(chunks_for_row)

In [183]:
len(all_general_chunks_recursive)

4752

In [185]:
df_general_chunks_recursive = pd.DataFrame(all_general_chunks_recursive)

In [ ]:
'''all_general_chunks_semantic = []
for index, row in df_final.iterrows():
    #  Вызываем нашу функцию для получения списка чанков из одной строки
    chunks_for_row = create_general_chunks(row, splitter=text_splitter_semantic)
    #  Добавляем все чанки из текущей строки в общий список
    all_general_chunks_semantic.extend(chunks_for_row)'''

In [ ]:
'''df_general_chunks_semantic = pd.DataFrame(all_general_chunks_semantic)'''

In [187]:
all_general_chunks_fusion = []
for index, row in df_final.iterrows():
    #  Вызываем нашу функцию для получения списка чанков из одной строки
    chunks_for_row = create_general_chunks(row, splitter=fusion_splitter)
    #  Добавляем все чанки из текущей строки в общий список
    all_general_chunks_fusion.extend(chunks_for_row)

In [189]:
df_general_chunks_fusion = pd.DataFrame(all_general_chunks_fusion)

In [191]:
len(all_general_chunks_fusion)

3696

In [ ]:
'''all_general_chunks_hyde = []
for index, row in df_final.iterrows():
    #  Вызываем нашу функцию для получения списка чанков из одной строки
    chunks_for_row = create_general_chunks(row, splitter=hyde_splitter)
    #  Добавляем все чанки из текущей строки в общий список
    all_general_chunks_hyde.extend(chunks_for_row)'''

In [ ]:
'''df_general_chunks_hyde = pd.DataFrame(all_general_chunks_hyde)'''

In [ ]:
'''len(all_general_chunks_hyde)'''

In [193]:
df_general_chunks_fusion['rag_chunk'][0]

'СВОДКА ВАКАНСИИ: Вакансия: Data Scientist в команду антифрода. Город:  Москва. Опыт: От 3 до 6 лет. График: Полный день. Занятость: Полная занятость. Минимальная зарплата: 0.0.Требования: имеете опыт решения различных data science задач использованием python имеете опыт решении практических задач связанных nlp. Обязанности: Улучшать алгоритмы поиска фрода и спама в сервисе, который обрабатывает тысячи событий в минуту. Искать нетривиальные и эффективные решения реальных...'

In [195]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

In [197]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name);

In [198]:
def df_to_langchain_documents(df):
    """Преобразует DataFrame чанков в список объектов LangChain Document."""
    documents = []
    for index, row in df.iterrows():
        # Важно: Сохраняем ID, чтобы можно было отследить источник
        # и сохраняем тип чанка, чтобы различать сводку и детали.
        documents.append(
            Document(
                page_content=row['rag_chunk'],
                metadata={
                    "id": row['id']
                }
            )
        )
    return documents

In [201]:
# --- A. База для РЕКУРСИВНОГО чанкования (Base RAG) ---
recursive_docs = df_to_langchain_documents(df_general_chunks_recursive)
db_recursive = FAISS.from_documents(recursive_docs, embeddings)

In [ ]:
# --- B. База для СЕМАНТИЧЕСКОГО чанкования (для сравнения) ---
'''semantic_docs = df_to_langchain_documents(df_general_chunks_semantic)
db_semantic = FAISS.from_documents(semantic_docs, embeddings)'''

In [ ]:
# --- C. База для HYDE ---
# HyDE использует свой набор чанков.
'''hyde_docs = df_to_langchain_documents(df_general_chunks_hyde)
db_hyde = FAISS.from_documents(hyde_docs, embeddings)'''

In [203]:
# --- D. База для FUSION RETRIEVAL (BM25 + FAISS) ---
# Fusion требует два ретривера, но оба используют ОДИН И ТОТ ЖЕ набор чанков.
fusion_docs = df_to_langchain_documents(df_general_chunks_fusion)
db_fusion_dense = FAISS.from_documents(fusion_docs, embeddings) # Dense Retriever (Faiss)
# BM25 Retriever (Sparse) будет создан из fusion_docs, используя специальный класс LangChain.

In [205]:
import pyarrow.parquet
df_general_chunks_fusion.to_parquet('df_fusion_chunks.parquet', index=False)

In [207]:
# 1. База для РЕКУРСИВНОГО чанкования
index_path_recursive = "./faiss_index_recursive2"
db_recursive.save_local(index_path_recursive)
print(f"База данных Recursive сохранена в: {index_path_recursive}")

База данных Recursive сохранена в: ./faiss_index_recursive2


In [ ]:
# 2. База для СЕМАНТИЧЕСКОГО чанкования
'''index_path_semantic = "./faiss_index_semantic"
db_semantic.save_local(index_path_semantic)
print(f"База данных Semantic сохранена в: {index_path_semantic}")'''

In [ ]:
# 3. База для HYDE
'''index_path_hyde = "./faiss_index_hyde"
db_hyde.save_local(index_path_hyde)
print(f"База данных HyDE сохранена в: {index_path_hyde}")'''

In [209]:
# 4. База для FUSION RETRIEVAL (Dense Index)
index_path_fusion = "./faiss_index_fusion_2"
db_fusion_dense.save_local(index_path_fusion)
print(f"База данных Fusion Dense сохранена в: {index_path_fusion}")

База данных Fusion Dense сохранена в: ./faiss_index_fusion_2
